In [1]:
:tags: [thebe-init]
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

SyntaxError: invalid syntax (1409200915.py, line 1)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def ad_as_shock_sim(pi_e=2.0, pi_star=2.0, Y_bar=100, r_bar=2.0,
                    phi_pi=1.5, lamb=0.3, alpha=5,
                    shock_type='None', shock_mag=5.0, T=25):

    T = int(T)
    pi = np.zeros(T)
    Y = np.zeros(T)
    r = np.zeros(T)

    pi[0] = pi_e
    Y[0] = Y_bar

    eps_AD = np.zeros(T)
    eps_AS = np.zeros(T)

    # Apply shock in period 2
    if shock_type == "Negative Demand":
        eps_AD[2] = -shock_mag
    elif shock_type == "Positive Supply":
        eps_AS[2] = -shock_mag
    elif shock_type == "Negative Supply":
        eps_AS[2] = shock_mag
    elif shock_type == "Expectations":
        pi[0] += shock_mag

    for t in range(1, T):
        # Policy rule (Taylor)
        r[t] = r_bar + phi_pi * (pi[t-1] - pi_star)

        # IS curve with demand shock
        Y[t] = Y_bar - alpha * (r[t] - r_bar) + eps_AD[t]

        # AS curve with supply shock
        pi[t] = pi_e + lamb * (Y[t] - Y_bar) + eps_AS[t]

    # Plots
    fig, axs = plt.subplots(1, 2, figsize=(13, 5))

    axs[0].plot(range(T), Y, label='Output (Y)', color='royalblue')
    axs[0].axhline(Y_bar, color='gray', linestyle='--')
    axs[0].set_title('Output Over Time')
    axs[0].set_xlabel('Time')
    axs[0].legend()
    axs[0].grid(True)

    axs[1].plot(range(T), pi, label='Inflation (π)', color='crimson')
    axs[1].axhline(pi_star, color='gray', linestyle='--')
    axs[1].set_title('Inflation Over Time')
    axs[1].set_xlabel('Time')
    axs[1].legend()
    axs[1].grid(True)

    plt.suptitle(f'💥 AD-AS Shock Simulation: {shock_type}', fontsize=14)
    plt.tight_layout()
    plt.show()

interact(ad_as_shock_sim,
         pi_e=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='πᵉ'),
         pi_star=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='π*'),
         Y_bar=FloatSlider(value=100, min=80, max=120, step=1, description='Ȳ'),
         r_bar=FloatSlider(value=2.0, min=0, max=5, step=0.1, description='r̄'),
         phi_pi=FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description='ϕ_π'),
         lamb=FloatSlider(value=0.3, min=0.05, max=1.0, step=0.05, description='λ'),
         alpha=FloatSlider(value=5, min=1, max=10, step=1, description='IS Sensitivity α'),
         shock_type=['None', 'Negative Demand', 'Positive Supply', 'Negative Supply', 'Expectations'],
         shock_mag=FloatSlider(value=5.0, min=0.0, max=10.0, step=0.5, description='Shock Size'),
         T=FloatSlider(value=25, min=10, max=50, step=1, description='Periods'));

interactive(children=(FloatSlider(value=2.0, description='πᵉ', max=5.0), FloatSlider(value=2.0, description='π…

# 💥 AD-AS Shocks and Dynamic Adjustment

This model shows how different macroeconomic **shocks** affect the short-run:



# 🧠 Core Equations

- **Aggregate Supply (AS):**
\[
\pi_t = \pi^e + \lambda (Y_t - \bar{Y}) + \varepsilon^{AS}
\]

- **IS Curve:**
\[
Y_t = \bar{Y} - \alpha (r_t - \bar{r}) + \varepsilon^{AD}
\]

- **Monetary Policy Rule (Taylor):**
\[
r_t = \bar{r} + \phi_\pi (\pi_t - \pi^*)
\]



Each period, the economy reacts, adjusts, and returns to the steady state.

We simulate this dynamically over **T periods**.